In [23]:
import duckdb
import pandas as pd
import networkx as nx
import datetime
import time
import requests
import csv

In [24]:
#df = pd.read_csv("C:/Users/caca_/Desktop/Gowalla_totalCheckins.csv", sep="	", nrows=225)
#df.columns = ['id_user', 'time', 'latitude', 'longitude' ,'location_id']

data = pd.read_csv("C:/Users/caca_/Desktop/Gowalla_totalCheckins.csv", sep="	")
data.columns = ['id_user', 'time', 'latitude', 'longitude' ,'location_id']

In [25]:
#Buscando o id de todos os usuários com registros no dia do desastre - 01/05/2010 e 02/05/2010

#res = duckdb.sql("SELECT COUNT(column0) FROM data WHERE STRFTIME('%Y-%m-%dT%H:%M:%S', column1) LIKE '2010-05-01T%' OR STRFTIME('%Y-%m-%dT%H:%M:%S', column1) LIKE '2010-05-02T%'") 
res = duckdb.sql("SELECT column0 FROM data WHERE STRFTIME('%Y-%m-%dT%H:%M:%S', column1) LIKE '2010-05-01T%' OR STRFTIME('%Y-%m-%dT%H:%M:%S', column1) LIKE '2010-05-02T%'").fetchall()

lista_ids = set(row[0] for row in res)
print(len(lista_ids))


BinderException: Binder Error: Referenced column "column1" not found in FROM clause!
Candidate bindings: "data.time"

In [16]:
#Determinando os usuarios que estiveram no Tennessee na data de interesse

import geopandas as gpd
from shapely.geometry import Point

tennessee = gpd.read_file('C:/Users/caca_/Desktop/tennessee.geojson')
users = []
cont = 0

for i in range(len(data)):

    latitude = data['latitude'][i]
    longitude = data['longitude'][i]

    point = Point(longitude, latitude)

    # Verificar se o ponto está dentro do polígono do Tennessee
    is_inside = point.within(tennessee.geometry.iloc[0])

    if is_inside:
        users.append(data.iloc[i].tolist())

print(len(users))

55891


In [22]:
with open('arquivo.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(users)

In [31]:
id = []
time = []
lat = []
long = []
places = []

users = pd.read_csv("usuarios_tennessee.csv", nrows=500)


for i in range(len(users)):
    query = f"SELECT * FROM data WHERE column0 = {i}"
    result = duckdb.sql(query).fetchall()

    for j in range(len(result)):
        id.append(result[j][0])
        time.append(result[j][1]) 
        lat.append(result[j][2]) 
        long.append(result[j][3])
        places.append(result[j][4])

new_data = {"id" : id,
        "timestamp" : time,
        "latitude" : lat,
        "longitude" : long,
        "place" : places
}
df = pd.DataFrame(new_data)
df.to_csv('arquivo_final.csv', index=False)
